# DHF - Lab 6 - Mathematical LOB modellingFichier

*Pierre ARTIGALA - Léon CHATAIGNAULT*
<a id='top'></a>

## Table of contents

### [0. Model](#q0)
### [1. Sanity check](#q1)
### [2. Main analysis](#q2)


## Imports

In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

<a id='q0'></a>

## 0. Model [[^]](#top)

Modeling assumptions The model used in this lab is a variant the Poisson models presented above. Price is constrained on a fixed grid $\{0, ..., N−1\}$. By convention, bid quantities are negative, ask quantities are positive. All orders are supposed to be of size 1. The limit order book is thus a vector X of size N with signed integers values.

In [2]:
def model_lob(
    N: int,
    K: int,
    mu: float,
    lambd: float,
    theta: float,
    X_0: np.array,
    T: float,
):
    """Run a simulation of the LOB model."""
    # Initialization
    t = 0
    X = X_0.copy()
    # Matrix in which the LOB history will be saved
    data = [X_0]
    l_delta = [0]
    # Iterations to modify the LOB
    while t < T:
        # Compute the bid and ask prices
        p_bid = np.where(X<0)[0].max()
        p_ask = np.where(X>0)[0].min()
        # Set the active windows (A for ask and B for bid)
        id_A = np.arange(p_bid+1, p_bid+K+1)
        id_B = np.arange(p_ask-K, p_ask)
        B = X[id_B]
        A = X[id_A]
        # Set the intensity vector
        norm_A = np.abs(A).sum()
        norm_B = np.abs(B).sum()
        intensity = np.array([mu, mu, K*lambd, K*lambd, theta*norm_B, theta*norm_A])
        # Draw delta the time interval to the next event (exponential with rate norm(intensity))
        norm_intensity = np.abs(intensity).sum()
        delta = np.random.exponential(scale = 1/norm_intensity)
        # Decide the type of the next event according to the intensity vector
        proba = intensity / norm_intensity
        next_event = np.random.choice(np.arange(0, len(intensity)), p=proba)
        # Modify the LOB according to the event
        if next_event == 0:
            # Process a bid market order
            X[p_bid] +=1
        elif next_event == 1:
            # Process an ask market order
            X[p_ask] -=1
        elif next_event == 2:
            # Process a bid limit order
            # Choose a price in B with uniform distribution
            p_event = np.random.choice(id_B)
            # Add a bid limit order at this price
            X[p_event] -= 1
        elif next_event == 3:
            # Process an ask limit order
            # Choose a price in A with uniform distribution
            p_event = np.random.choice(id_A)
            # Add an ask limit order at this price
            X[p_event] += 1
        elif next_event == 4:
            # Process a bid cancellation
            # Choose an order to be cancelled with probability proportional to the coordinates of B
            proba = np.abs(B)/norm_B
            p_event = np.random.choice(id_B, p=proba)
            # Cancel an order at the chosen price
            X[p_event] += 1
        else:
            # Process an ask cancellation
            # Choose an order to be cancelled with probability proportional to the coordinates of A
            proba = np.abs(A)/norm_A
            p_event = np.random.choice(id_A, p=proba)
            # Cancel an order at the chosen price
            X[p_event] -= 1
        t += delta
        # Append the current state to the dataframe (if t<=T)
        if t <= T:
            l_delta.append(t)
            data.append(X.copy())
    # Create a dataframe to save the LOB history
    df = pd.DataFrame(data=data, index=l_delta)
    # Save the dataframe
    df.to_csv("results_lob.csv", index=False)
    return df

def load_results():
    """Load the .csv file."""
    df = pd.read_csv("results_lob.csv")
    return df

In [3]:
N = 1000
K = 10
mu = 2.5
lambd = 1
theta = 0.2
X_0 = np.array([-5 for _ in range(int(N/2))] + [5 for _ in range(int(N/2))])
T = 10**2

df = model_lob(
    N=N,
    K=K,
    mu=mu,
    lambd=lambd,
    theta=theta,
    X_0=X_0,
    T=T
)

In [4]:
df = load_results()
df

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,...,5,5,5,5,5,5,5,5,5,5
1,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,...,5,5,5,5,5,5,5,5,5,5
2,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,...,5,5,5,5,5,5,5,5,5,5
3,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,...,5,5,5,5,5,5,5,5,5,5
4,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,...,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4053,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,...,5,5,5,5,5,5,5,5,5,5
4054,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,...,5,5,5,5,5,5,5,5,5,5
4055,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,...,5,5,5,5,5,5,5,5,5,5
4056,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,...,5,5,5,5,5,5,5,5,5,5


<a id='q1'></a>

## 1. Sanity check [[^]](#top)

### 1.1 Print a small extract of the csv output of the simulation to illustrate that the orders are correctly processed.

### 1.2 Check that the number of bid and ask market and limit orders in the simulation is correct (in agreement with the intensity parameters) and that the placement of limit orders in the simulation is correct (uniform distribution).

<a id='q2'></a>

## 2. Main analysis [[^]](#top)

### 2.1 Plot the average shape of the simulated LOB. Compute an approximation of the value of the average shape away from the best bid and ask, using a queuing model. Are your simulations in agreement with the analytical result ?

### 2.2 Plot the spread distribution.

### 2.3 Plot the mid-price variation at a large scale.

### 2.4 Plot the distribution of mid-price increments for several sampling periods. Comment.

### 2.5 Plot the autocorrelation function of the mid-price increments. Comment.

### 2.6 Plot the variance of mid-price increments as a function of the sampling period. Comment.